# Introduction
<center><img src="https://i.hizliresim.com/50kvrvm.png" alt="Görsel" style="max-width: 100%; height: 600px;"> <center>
    <br>

<center> <b> It attempts to solve the problem of recognizing gender from human faces using Convolutional Neural Networks (CNN),EfficientNetB3,VGG16 and ResNet50 methods. <b><center>

# Python Libraries


In [ ]:
import os  # İşletim sistemi işlemleri için
import itertools  # İterasyon işlemleri için

import numpy as np  # Sayısal hesaplamalar için
import pandas as pd  # Veri analizi ve veri manipülasyonu için
import seaborn as sns  # Veri görselleştirme için

sns.set_style('darkgrid')  # Seaborn stil ayarları
import matplotlib.pyplot as plt  # Veri görselleştirme için
from sklearn.model_selection import train_test_split  # Veri kümesini eğitim ve test setlerine ayırmak için
from sklearn.metrics import confusion_matrix, classification_report  # Performans değerlendirme metrikleri için

import tensorflow as tf  # Derin öğrenme modelleri için
from tensorflow import keras  # TensorFlow'un Keras API'si
from tensorflow.keras.models import Sequential  # Sıralı model oluşturmak için
from tensorflow.keras.optimizers import Adam, Adamax  # Optimizasyon algoritmaları için
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Görüntü verisi üretmek ve artırmak için
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization  # Katmanlar ve katmanlar arası işlemler için
from tensorflow.keras.models import Model  # Genel model sınıfı
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau  # Eğitim sürecini kontrol eden geri aramalar (callbacks) için
from tensorflow.keras.applications import EfficientNetB3  # EfficientNetB3 önceden eğitilmiş modeli için
from keras.preprocessing import image  # Görüntü işleme için
from tensorflow.keras.utils import load_img  # Görüntü yükleme için
from tensorflow.keras.preprocessing.image import load_img  # Görüntü yükleme için
from keras.applications import VGG16  # VGG16 önceden eğitilmiş modeli için
from tensorflow.keras.applications import ResNet50  # ResNet50 önceden eğitilmiş modeli için

# Uyarıları yok saymak için
import warnings
warnings.filterwarnings('ignore')

# Read and Analyse Data

In [ ]:
def loading_the_data(data_dir):
    filepaths = []
    labels = []

    filelist = os.listdir(data_dir)

    for file in filelist:
        filename = os.path.join(data_dir, file)
        temp = filename.split('_')
        gender = int(temp[1])
        filepaths.append(filename)
        labels.append(gender)
        

    Fseries = pd.Series(filepaths, name='filepaths')
    Lseries = pd.Series(labels, name='labels')

    df = pd.concat([Fseries, Lseries], axis=1)
    
    return df

In [ ]:
data_dir = '/kaggle/input/utkface-new/UTKFace'
df = loading_the_data(data_dir)
df['labels'] = df['labels'].apply(lambda x: 'male' if x == 0 else 'female')
df

In [ ]:
data_balance = df.labels.value_counts()
data_balance

In [ ]:
#bir pasta grafiği (pie chart) üzerindeki dilimlerin yüzdelik ve toplam sayısal değerlerini göstermek için kullanılır. 
def custom_autopct(pct):
    total = sum(data_balance)
    val = int(round(pct*total/100.0))
    return "{:.1f}%\n({:d})".format(pct, val)

In [ ]:
plt.pie(data_balance, labels = data_balance.index, autopct=custom_autopct, colors = ["#0000FF", "#FF69B4"])
plt.title("Data balance")
plt.axis("equal")
plt.show()

# Helper Functions


In [ ]:
def model_performance(history, Epochs):
    # Eğitim sürecinde her epoch için elde edilen eğitim doğruluğunu (accuracy) alır
    tr_acc = history.history['accuracy']

    # Eğitim sürecinde her epoch için elde edilen eğitim kaybını (loss) alır
    tr_loss = history.history['loss']

    # Eğitim sürecinde her epoch için elde edilen doğrulama doğruluğunu (validation accuracy) alır
    val_acc = history.history['val_accuracy']

    # Eğitim sürecinde her epoch için elde edilen doğrulama kaybını (validation loss) alır
    val_loss = history.history['val_loss']
    
    Epochs = [i+1 for i in range(len(tr_acc))]
    
    plt.figure(figsize= (20, 8))
    plt.style.use('fivethirtyeight')
    
    plt.subplot(1, 2, 1)
    plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
    plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
    plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout
    plt.show()

In [ ]:
#Doğrulama ve test veri setleri kullanılarak modelin performansını değerlendirir. 
#model.evaluate metodu, belirtilen veri setine göre modelin kaybını (loss) ve metriklerini (örneğin doğruluk) hesaplar. 
def model_evaluation(model):
    train_score = model.evaluate(train_gen, verbose= 1)
    valid_score = model.evaluate(valid_gen, verbose= 1)
    test_score = model.evaluate(test_gen, verbose= 1)
    
    print("Train Loss: ", train_score[0])
    print("Train Accuracy: ", train_score[1])
    print('-' * 20)
    print("Validation Loss: ", valid_score[0])
    print("Validation Accuracy: ", valid_score[1])
    print('-' * 20)
    print("Test Loss: ", test_score[0])
    print("Test Accuracy: ", test_score[1])

In [ ]:
def get_pred(model, test_gen):
    
    preds = model.predict(test_gen)
    y_pred = np.argmax(preds, axis = 1) #np.argmax,içerisindeki en büyük değerin indeksini döndüren bir fonksiyondur.
    
    return y_pred

In [ ]:
def plot_confusion_matrix(test_gen, y_pred):
    
    g_dict = test_gen.class_indices
    classes = list(g_dict.keys())
    
    cm = confusion_matrix(test_gen.classes, y_pred)

    plt.figure(figsize= (10, 10))
    plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation= 45, fontsize=8)  
    plt.yticks(tick_marks, classes)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')
    
    
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    plt.show()

In [ ]:
# Konvolüsyonel (evrişimsel) katman bloğu oluşturan bir fonksiyon tanımlanıyor
# 'filters' parametresi ile evrişimsel katmanlardaki filtre sayısı, 'act' parametresi ile aktivasyon fonksiyonu belirleniyor
def conv_block(filters, act='relu'):
    
    # Sıralı bir model (Sequential) oluşturuluyor. Bu model, katmanların birbirini takip ettiği bir yapıdadır.
    block = Sequential()

    # İlk evrişimsel (Conv2D) katman ekleniyor. 
    # 'filters' ile belirlenen sayıda filtre kullanılıyor, her bir filtrenin boyutu 3x3.
    # 'activation' parametresi ile aktivasyon fonksiyonu belirleniyor (varsayılan olarak ReLU).
    # 'padding' parametresi 'same' olarak ayarlandığından, giriş ile aynı boyutta bir çıktı üretilir.
    block.add(Conv2D(filters, 3, activation=act, padding='same'))

    # İkinci evrişimsel (Conv2D) katman ekleniyor. Yine 'filters' ile belirlenen sayıda 3x3 filtre kullanılıyor.
    # Aktivasyon fonksiyonu ve padding ayarları aynı şekilde uygulanıyor.
    block.add(Conv2D(filters, 3, activation=act, padding='same'))

    # Batch normalization katmanı ekleniyor. Bu katman, mini-batch normalizasyonu uygulayarak eğitimi hızlandırır ve kararlılığını artırır.
    block.add(BatchNormalization())

    # Max pooling katmanı ekleniyor. Bu katman, uzaysal boyutları küçültür ve modelin hesaplama maliyetini azaltır.
    # Varsayılan olarak, 2x2 boyutunda bir pencere kullanarak maksimum değerleri alır.
    block.add(MaxPooling2D())
    
    # Oluşturulan bloğu döndürüyor.
    return block

In [ ]:
# Tam bağlantılı (dense) katman bloğu oluşturan bir fonksiyon tanımlanıyor
# 'units' parametresi ile dense katmandaki nöron sayısı, 'dropout_rate' parametresi ile dropout oranı, 'act' parametresi ile aktivasyon fonksiyonu belirleniyor
def dense_block(units, dropout_rate, act='relu'):
    
    # Sıralı bir model (Sequential) oluşturuluyor. Bu model, katmanların birbirini takip ettiği bir yapıdadır.
    block = Sequential()

    # Tam bağlantılı (Dense) katman ekleniyor.
    # 'units' ile belirlenen sayıda nöron kullanılıyor.
    # 'activation' parametresi ile aktivasyon fonksiyonu belirleniyor (varsayılan olarak ReLU).
    block.add(Dense(units, activation=act))

    # Batch normalization katmanı ekleniyor. Bu katman, mini-batch normalizasyonu uygulayarak eğitimi hızlandırır ve kararlılığını artırır.
    block.add(BatchNormalization())

    # Dropout katmanı ekleniyor. Bu katman, belirtilen oranda (dropout_rate) rastgele nöronları devre dışı bırakarak aşırı öğrenmeyi (overfitting) önler.
    block.add(Dropout(dropout_rate))
    
    # Oluşturulan bloğu döndürüyor.
    return block

In [ ]:
from tensorflow.keras.preprocessing import image
def manuel_gender_test(model):
    # Görseli yükleme ve işleme fonksiyonu
    def load_and_process_image(image_path, img_size=(224, 224)):
        img = image.load_img(image_path, target_size=img_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Görseli yeniden ölçeklendirin
        return img_array
    image_paths = [
    "/kaggle/input/utkface-new/crop_part1/101_1_2_20170105174739309.jpg.chip.jpg",
    "/kaggle/input/utkface-new/crop_part1/14_0_0_20170110225712028.jpg.chip.jpg",
    "/kaggle/input/utkface-new/crop_part1/11_1_4_20170109201611941.jpg.chip.jpg",
    "/kaggle/input/utkface-new/crop_part1/10_0_0_20170110220539329.jpg.chip.jpg",
    "/kaggle/input/utkface-new/crop_part1/10_0_0_20170110225444491.jpg.chip.jpg",
    "/kaggle/input/utkface-new/crop_part1/10_1_0_20170103175323250.jpg.chip.jpg",
    "/kaggle/input/utkface-new/crop_part1/10_1_0_20170109204844109.jpg.chip.jpg",
    "/kaggle/input/utkface-new/crop_part1/10_0_0_20170110220546177.jpg.chip.jpg"
]



    # Sınıf isimlerini burada belirtin
    class_names = ['famale', 'male']

    # Tahminleri saklamak için bir liste oluşturun
    predictions = []

    # Görsel yollarını döngü ile işleyip tahmin yapın
    for img_path in image_paths:
        processed_image = load_and_process_image(img_path)
        prediction = model.predict(processed_image)
        predicted_class = np.argmax(prediction, axis=1)[0]
        predicted_gender = class_names[predicted_class]

        # Tahmin sonuçlarını saklayın
        predictions.append({
            'image_path': img_path,
            'predicted_gender': predicted_gender
        })

    # Tahmin sonuçlarını DataFrame'e dönüştürün
    df_predictions = pd.DataFrame(predictions)



    # Görselleştirme
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))

    for ax, prediction in zip(axes.flatten(), predictions):
        img = image.load_img(prediction['image_path'])
        ax.imshow(img)
        ax.set_title(f"Tahmin edilen cinsiyet: {prediction['predicted_gender']}")
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# Train - Test Split

In [ ]:
train_df, ts_df = train_test_split(df, train_size = 0.8, shuffle = True, random_state = 42)
#shuffle: Veri kümesinin karıştırılıp karıştırılmayacağını belirler.(Kendi içindeki sıralaması)
#random_state:Rastgele karıştırma işleminin tekrarlanabilirliğini sağlar.


valid_df, test_df = train_test_split(ts_df, train_size = 0.5, shuffle = True, random_state = 42)

In [ ]:
#Batch size, her bir eğitim iterasyonunda veya tahmin işleminde modele aynı anda beslenen veri örneklerinin sayısıdır.
batch_size = 16

# Eğitim ve test için kullanılacak görüntülerin boyutlarını belirler.
img_size = (224, 224)

# ImageDataGenerator sınıfı, veri artırma (data augmentation) ve veri ön işleme için kullanılır.
# Burada, görüntüleri 0-255 aralığından 0-1 aralığına yeniden ölçeklendirmek için 'rescale' parametresi kullanılıyor.
tr_gen = ImageDataGenerator(rescale=1. / 255)
ts_gen = ImageDataGenerator(rescale=1. / 255)


train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'filepaths', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= False, batch_size= batch_size)

In [ ]:
g_dict = train_gen.class_indices     
classes = list(g_dict.keys())       
images, labels = next(train_gen)      

# ploting the patch size samples
plt.figure(figsize= (20, 20))

for i in range(batch_size):
    plt.subplot(4, 4, i + 1)
    image = images[i]
    plt.imshow(image)
    index = np.argmax(labels[i])  # get image index
    class_name = classes[index]   # get class of image
    plt.title(class_name, color= 'black', fontsize= 16)
    plt.axis('off')
plt.tight_layout()
plt.show()

# Model Buildings

In [ ]:
#Görüntü boyutu, modeller 224x224 boyutlarda başarılı olduğu için bu boyut kullanılmıştır.
img_size = (224, 224)

# Görüntülerin renk kanallarının sayısını belirtir.
channels = 3

# Görüntülerin boyutunu ve renk kanalı sayısını belirtir.
img_shape = (img_size[0], img_size[1], channels)

#sınıf sayısını belirtir.
class_counts = len(list(train_gen.class_indices.keys()))

# CNN Model Building

In [ ]:
cnn_model = Sequential()

# 16 filtreli, 3x3 boyutlu kernel kullanarak bir Conv2D katmanı eklenir.
# "same" padding kullanılır, giriş boyutu ile çıkış boyutu aynı olur.
# ReLU aktivasyon fonksiyonu kullanılır.
# input_shape, ilk katman için görüntülerin şeklini belirtir.
cnn_model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu", input_shape= img_shape))

#BatchNormalization, aktivasyonları normalize ederek ağın daha hızlı ve daha istikrarlı bir şekilde eğitilmesine yardımcı olur.
# Ağırlıkların ve biasların normalleştirilmesini sağlar.
cnn_model.add(BatchNormalization())

#Giriş boyutunu yarıya indirir, önemli özelliklerin öğrenilmesini sağlar.
cnn_model.add(MaxPooling2D())

# 32 filtreli bir Conv2D katmanı eklenir.
cnn_model.add(conv_block(32))

cnn_model.add(conv_block(64))

cnn_model.add(conv_block(128))

cnn_model.add(conv_block(256))

cnn_model.add(conv_block(512))

cnn_model.add(Flatten())

cnn_model.add(dense_block(256, 0.5))

cnn_model.add(dense_block(128, 0.3))

cnn_model.add(dense_block(64, 0.2))

cnn_model.add(dense_block(32, 0.2))


# Çıkış katmanı olarak sınıf sayısı kadar birimli bir Dense katman eklenir.
# Softmax aktivasyon fonksiyonu, sınıf olasılıklarını hesaplamak için kullanılır
cnn_model.add(Dense(class_counts, activation = "softmax"))

# Modelin derlenmesi.
# Çoklu sınıf sınıflandırması için categorical_crossentropy kaybı kullanılır.
# Adam optimizer kullanılır.
# Doğruluk (accuracy) metriği izlenir.
cnn_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [ ]:
epochs = 10   

cnn_history = cnn_model.fit(train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, shuffle= False)

#verbose=1, her epoch sonunda ilerleme çubuğu ve kayıp metriklerinin yazdırılmasını sağlar.
#shuffle, eğitim ve doğrulama veri akışlarının sıralı olacağını ve her epoch öncesi karıştırılmayacağını belirtir.

# CNN Model Performance - Prediction

In [ ]:
model_performance(cnn_history, epochs)

In [ ]:
model_evaluation(cnn_model)

model_evaluation(cnn_model)

In [ ]:
cnn_y_pred = get_pred(cnn_model, test_gen)

plot_confusion_matrix(test_gen, cnn_y_pred)

# CNN Model Manuel Prediction

In [ ]:
from tensorflow.keras.preprocessing import image

manuel_gender_test(cnn_model)

# EfficientNetB3 Model Building


In [ ]:
#weights Önceden eğitilmiş ağırlıkların hangi kaynaktan yükleneceğini belirtir. 
#include_top Modelin üst katmanının (fully connected layer) dahil edilip edilmeyeceğini belirtir. False olarak ayarlandığında, modelin üst katmanı dahil edilmez.
#pooling modelin tüm özellik haritalarını kullanarak çıkış üreteceği anlamına gelir.
EffNetB3_base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape = img_shape, pooling= None)

EffNetB3_x = EffNetB3_base_model.output
#Global Average Pooling, her bir özellik haritasındaki tüm özelliklerin ortalamasını alır ve tek bir özellik vektörü oluşturur. 
EffNetB3_x = GlobalAveragePooling2D()(EffNetB3_x)
EffNetB3_x = BatchNormalization()(EffNetB3_x)
EffNetB3_x = dense_block(128, 0.5)(EffNetB3_x)
EffNetB3_x = dense_block(32, 0.2)(EffNetB3_x)
EffNetB3_predictions = Dense(class_counts, activation = "softmax")(EffNetB3_x)    


EfficientNetB3_model = Model(inputs = EffNetB3_base_model.input, outputs = EffNetB3_predictions)

In [ ]:
EfficientNetB3_model.compile(optimizer=Adamax(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

EfficientNetB3_model.summary()

In [ ]:
epochs = 10

EfficientNetB3_history = EfficientNetB3_model.fit(train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, shuffle= False)

# EfficientNetB3 Model Performance - Prediction

In [ ]:
model_performance(EfficientNetB3_history, epochs)

In [ ]:
model_evaluation(EfficientNetB3_model)

In [ ]:
EfficientNetB3_y_pred = get_pred(EfficientNetB3_model, test_gen)


plot_confusion_matrix(test_gen, EfficientNetB3_y_pred)

# EfficientNetB3 Model Manuel Prediction

In [ ]:
from tensorflow.keras.preprocessing import image

manuel_gender_test(EfficientNetB3_model)

# VGG16  Model Building

In [ ]:
VGG16_base_model = VGG16(weights='imagenet', include_top=False, input_shape = img_shape, pooling= 'max')
# Maksimum pooling işlemi, her bir özellik haritasındaki en büyük değeri seçerek boyutları azaltır.

#modelin önceden eğitilmiş kısmını dondurmak ve yalnızca yeni eklenen katmanların eğitilmesini sağlamak için ilk 15 katman eğitilmesin.
for layer in VGG16_base_model.layers[:15]:
    layer.trainable = False
    
    

VGG16_x = VGG16_base_model.output
VGG16_x = Flatten()(VGG16_x)
VGG16_x = Dense(512, activation = 'relu')(VGG16_x)
VGG16_x = Dropout(0.2)(VGG16_x)   
VGG16_x = Dense(256, activation = 'relu')(VGG16_x)
VGG16_x = Dense(128, activation = 'relu')(VGG16_x)
VGG16_x = Dense(32, activation = 'relu')(VGG16_x)
VGG16_predictions = Dense(class_counts, activation = "sigmoid")(VGG16_x)    


VGG16_model = Model(inputs = VGG16_base_model.input, outputs = VGG16_predictions)

In [ ]:
for layer in VGG16_model.layers:
    print(layer.name, layer.trainable)

In [ ]:
VGG16_model.compile(optimizer=Adamax(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

VGG16_model.summary()

In [ ]:
epochs = 2   

VGG16_history = VGG16_model.fit(train_gen, epochs= epochs, verbose= 1, validation_data= valid_gen, shuffle= False)

# VGG16 Model Performance - Prediction

In [ ]:
model_performance(VGG16_history, epochs)

In [ ]:
model_evaluation(VGG16_model)

In [ ]:
VGG16_y_pred = get_pred(VGG16_model, test_gen)

plot_confusion_matrix(test_gen, VGG16_y_pred)

# VGG16 Model Manuel Prediction

In [ ]:
from tensorflow.keras.preprocessing import image
manuel_gender_test(VGG16_model)

# ResNet50 Model Building


In [ ]:
resnet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50_x = resnet50_base_model.output
resnet50_x = GlobalAveragePooling2D()(resnet50_x)
resnet50_x = Dense(512, activation='relu')(resnet50_x)
resnet50_x = Dropout(0.5)(resnet50_x)
resnet50_predictions = Dense(2, activation='softmax')(resnet50_x)

In [ ]:
resnet50_model = Model(inputs=resnet50_base_model.input, outputs=resnet50_predictions)
resnet50_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
resnet50_model.summary()

In [ ]:
epochs = 2

resnet50_history = resnet50_model.fit(train_gen,epochs=epochs,validation_data=valid_gen,verbose=1)

# ResNet50 Model Performance - Prediction

In [ ]:
model_performance(resnet50_history, epochs)

In [ ]:
model_evaluation(resnet50_model)

In [ ]:
resnet50_y_pred = get_pred(resnet50_model, test_gen)

plot_confusion_matrix(test_gen, resnet50_y_pred)

# ResNet50 Model Manuel Prediction

In [ ]:
from tensorflow.keras.preprocessing import image

manuel_gender_test(resnet50_model)

# Models Comparison

In [ ]:
def plot_accuracy_loss(models, model_names, train_gen, valid_gen, test_gen):
    train_accuracies = []
    valid_accuracies = []
    test_accuracies = []

    train_losses = []
    valid_losses = []
    test_losses = []

    for model, name in zip(models, model_names):
        train_score = model.evaluate(train_gen, verbose=1)
        valid_score = model.evaluate(valid_gen, verbose=1)
        test_score = model.evaluate(test_gen, verbose=1)

        train_losses.append(train_score[0])
        valid_losses.append(valid_score[0])
        test_losses.append(test_score[0])

        train_accuracies.append(train_score[1])
        valid_accuracies.append(valid_score[1])
        test_accuracies.append(test_score[1])

    plt.figure(figsize=(14, 7))
    plt.subplot(1, 2, 1)
    plt.plot(model_names, train_accuracies, 'o-', label='Train Accuracy')
    plt.plot(model_names, valid_accuracies, 'o-', label='Validation Accuracy')
    plt.plot(model_names, test_accuracies, 'o-', label='Test Accuracy')
    plt.title('Model Accuracy Comparison')
    plt.xlabel('Model')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(model_names, train_losses, 'o-', label='Train Loss')
    plt.plot(model_names, valid_losses, 'o-', label='Validation Loss')
    plt.plot(model_names, test_losses, 'o-', label='Test Loss')
    plt.title('Model Loss Comparison')
    plt.xlabel('Model')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

model_names = ['CNN', 'EfficientNetB3', 'VGG16', 'ResNet50']
models = [cnn_model, EfficientNetB3_model, VGG16_model, resnet50_model]

plot_accuracy_loss(models, model_names, train_gen, valid_gen, test_gen)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrices(models, model_names, test_gen):
    plt.figure(figsize=(20, 20))
    
    for i, (model, name) in enumerate(zip(models, model_names), 1):
        y_pred = model.predict(test_gen)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = test_gen.classes

        cm = confusion_matrix(y_true, y_pred_classes)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_gen.class_indices.keys())
        
        plt.subplot(2, 2, i)
        disp.plot(ax=plt.gca(), values_format='d')
        plt.title(f'Confusion Matrix for {name}')

    plt.tight_layout()
    plt.show()

plot_confusion_matrices(models, model_names, test_gen)